![Banner](images/banner.png)

# python-oracledb Introduction

This Jupyter Notebook shows how to use [python-oracledb](https://oracle.github.io/python-oracledb/) in its default 'Thin' mode that connects directly to Oracle Database.

# python-oracledb Connection

# Architecture

Documentation reference link: [Introduction to python-oracledb](https://python-oracledb.readthedocs.io/en/latest/user_guide/introduction.html)

![Architecture](images/architecture-thin.png)

# Installation

Documentation reference link: [python-oracledb Installation](https://python-oracledb.readthedocs.io/en/latest/user_guide/installation.html)

### **Install python-oracledb**

Install with a command like one of the following:

```
$ python3 -m pip install oracledb --upgrade
$ python3 -m pip install oracledb --upgrade --user
$ python3 -m pip install oracledb --upgrade --user --proxy=http://proxy.example.com:80
```

To use python-oracledb, your application code can import the module:

In [ ]:
import oracledb

# Connecting to a Database

**Connections are used for executing SQL and PL/SQL in an Oracle Database**

Documentation reference link: [Connecting to Oracle Database](https://python-oracledb.readthedocs.io/en/latest/user_guide/connection_handling.html)

In [ ]:
# Credentials
un = "pythondemo"
pw = "welcome"

Instead of hard coding the password, you could prompt for a value, pass it as an environment variable, or use Oracle "external authentication".

### Easy Connect Syntax: "hostname/servicename"

In [ ]:
cs = "localhost/orclpdb1"

connection = oracledb.connect(user=un, password=pw, dsn=cs)
print(connection)

Oracle Client 19c improved [Easy Connect Plus](https://www.oracle.com/pls/topic/lookup?ctx=dblatest&id=GUID-8C85D289-6AF3-41BC-848B-BF39D32648BA) syntax with additional optional settings, for example:

```
cs = "tcps://my.cloud.com:1522/orclpdb1?connect_timeout=4&expire_time=10"
```

<!-- See the [technical brief](https://download.oracle.com/ocomdocs/global/Oracle-Net-19c-Easy-Connect-Plus.pdf). -->

### Oracle Network and Oracle Client Configuration Files

Oracle Database's `tnsnames.ora` file can be used.  This file maps a connect descriptor to an alias.  

Documentation reference link: [Optional configuration files](https://python-oracledb.readthedocs.io/en/latest/user_guide/initialization.html#optional-oracle-net-configuration-files)

Your Python code could use the alias as the connection `dsn` value:
```
connection = oracledb.connect(user=un, password=pw, dsn="highperfdb", config_dir="/opt/oracle/configdir")
```

## Connection Types

### Standalone Connections

Standalone connections are simple to create.

![Stand-alone Connection](images/standalone-connection.png)

In [ ]:
# Stand-alone Connections

connection = oracledb.connect(user=un, password=pw, dsn=cs)

print(connection)

### Pooled Connections

#### Pools are highly recommended if you have:
- a lot of connections that will be used for short periods of time
- or a small number of connections that are idle for long periods of time

#### Pool advantages
- Reduced cost of setting up and tearing down connections
- Dead connection detection and automatic re-establishment

![Pooled connection](images/pooled-connection.png)

In [ ]:
# Pooled Connections

# Call once during application initization
pool = oracledb.create_pool(user=un, password=pw, dsn=cs,
                            min=1, max=10, increment=1)

# Get a connection when needed in the application body
with pool.acquire() as connection:
    # do_something_useful(connection)
    print(f"Got a connection to Oracle Database {connection.version}")

**Tip** Use a fixed size pool `min` = `max` and `increment = 0`.  See [Guideline for Preventing Connection Storms: Use Static Pools](https://www.oracle.com/pls/topic/lookup?ctx=dblatest&id=GUID-7DFBA826-7CC0-4D16-B19C-31D168069B54).

### Closing Connections

Close connections when not needed.  This is important for pooled connections.

```
connection.close()
```

To avoid resource closing order issues, you may want to use `with` or let resources be closed at end of scope:

```
with pool.acquire() as connection:
    do_something(connection)
```

## Database Resident Connection Pooling

**Connection pooling on the database tier**

Documentation reference link: [Database Resident Connection Pooling (DRCP)](https://python-oracledb.readthedocs.io/en/latest/user_guide/connection_handling.html#database-resident-connection-pooling-drcp)

Dedicated server processes are the default in the database, but DRCP is an alternative when the database server is short of memory.

![DRCP architecture](images/drcp-architecture.png)

Use DRCP if and only if:
- The database computer doesn't have enough memory for all the server processes for all open application connections
- When you have thousands of users which need access to a database server session for a short period of time
- Applications mostly use same database credentials, and have identical session settings

Using DRCP in conjunction with a python-oracledb connection pool is recommended.

#### Memory example with 5000 application users and a DRCP pool of size 100
![DRCP memory comparison](images/drcp-comparison.png)

In Python, the connect string must request a pooled server, for example with ':pooled'.  For best efficiency to allow database server session re-use, set a connection class and use the purity 'PURITY_SELF'.

```
pool = oracledb.create_pool(user=un, password=pw, dsn="dbhost.example.com/orclpdb1:pooled",
                            cclass="MYCLASS", purity=oracledb.PURITY_SELF)

connection = pool.acquire()
```

Don't forget to start the pool first!:
```
SQL> execute dbms_connection_pool.start_pool()
```

Note DRCP is pre-enabled on Oracle Autonomous Database.